# Week 2 — Yoneda Lemma (Micro‑Demo)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sridharmahadevan/Category-Theory-for-AGI-UMass-CMPSCI-692CT/blob/main/notebooks/week02_yoneda_microdemo.ipynb)
<br/>
_Replace `sridharmahadevan/Category-Theory-for-AGI-UMass-CMPSCI-692CT` above once you push this repo to GitHub._

### Environment (run first)
This pins a minimal, stable stack. GPU is **optional**; notebooks run on CPU.

In [ ]:
%%capture
# Core scientific stack + causal / graph tooling
%pip install -q numpy==1.* pandas==2.* matplotlib==3.* networkx==3.* pgmpy==0.1.* graphviz==0.20.*
# Torch CPU by default (Colab often preinstalls a GPU build; this is a safe fallback)
%pip install -q torch --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
# Install system graphviz only if available (Colab & many Linux envs). Safe to skip elsewhere.
!command -v apt-get >/dev/null && apt-get -y -qq install graphviz || echo "apt-get not available; skipping system graphviz"

In [ ]:
import platform, sys
print("Python:", platform.python_version())
try:
    import torch
    print("Torch:", torch.__version__, "| CUDA available?", torch.cuda.is_available())
    device = "cuda" if torch.cuda.is_available() else "cpu"
except Exception as e:
    print("Torch not installed, proceeding CPU-only.")
    device = "cpu"
device


## Learning goals
- See the **Yoneda construction** on the tiniest possible category with objects `A, B` and morphisms `{id_A, id_B, f: A→B}`.
- Given \(x ∈ F(A)\), construct the natural transformation \(τ_x : \mathrm{Hom}(-,A) ⇒ F\) by \(τ_x(h) = F(h)(x)\).


## Tiny category and a simple Set-valued functor F

In [ ]:

# Objects and morphisms
Ob = {"A", "B"}
morphisms = {
    ("A","A"): ["id_A"],
    ("B","B"): ["id_B"],
    ("A","B"): ["f"],
    ("B","A"): []  # no morphisms B→A in this toy
}

# Define a covariant functor F : C → Set
F = {
    "A": {"a0", "a1"},
    "B": {"b0", "b1", "b2"}
}

# Action on morphisms (functions between the sets)
def F_on(m, x):
    # m is a string in {"id_A","id_B","f"}
    if m == "id_A": return x
    if m == "id_B": return x
    if m == "f":
        # Choose a specific mapping F(f): F(A)→F(B)
        mapping = {"a0":"b1", "a1":"b2"}
        return mapping[x]
    raise ValueError("unknown morphism")


## Yoneda: element x ∈ F(A) ↦ natural transformation τ_x : Hom(-,A) ⇒ F

In [ ]:

# Hom(-,A) at object A is Hom(A,A) = {id_A}; at B it is Hom(B,A) = ∅ in this toy
Hom_to_A = {
    "A": ["id_A"],  # A→A
    "B": []         # B→A is empty here
}

def tau_x(x):
    # Returns components τ_x(A): Hom(A,A)→F(A) and τ_x(B): Hom(B,A)→F(B)
    comp_A = { "id_A": F_on("id_A", x) }
    comp_B = {}  # empty function (domain empty set)
    return {"A": comp_A, "B": comp_B}

x = "a0"
tau = tau_x(x)
print("τ_x(A) maps id_A ↦", tau["A"]["id_A"])
print("τ_x(B) is the unique empty function:", tau["B"])



### Now include the nontrivial Hom(A,B) and check naturality along f: A→B
Natural transformation condition for \(h: A→B\):  
\(F(h)(x) = τ_x(B)(h)\) when we view \(h ∈ \mathrm{Hom}(A,B) = \mathrm{Hom}(-,A)(B)\).


In [ ]:

# Extend Hom(-,A) at B by considering h = f: A→B as an element from A's hom-set perspective.
# In the (covariant) Yoneda setup we evaluate via F(f)(x).
x = "a1"
lhs = F_on("f", x)  # F(f)(x) ∈ F(B)
# τ_x(B)(f) is "apply F to f then x"—in this tiny encoding, that equals lhs by construction.
rhs = lhs
print("F(f)(x) =", lhs, "| τ_x(B)(f) =", rhs, "| equal?", lhs == rhs)


## Exercises

In [ ]:

# 1) Change mapping in F_on for 'f' and re-run; confirm τ_x changes accordingly.
# 2) Add a new object C and a morphism g: B→C; extend F and verify naturality squares commute.
# 3) Try a different functor G and build τ_x for G; compare with F.
